## Evaluation of creep compliances from FEM model

### Imports and working path

In [ ]:
# --------------------------------------
# Imports
# --------------------------------------
from math import *
import numpy as np
import pandas as pd
import os
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from itertools import cycle
from pathlib import Path
from matplotlib.lines import Line2D

import nest_asyncio
nest_asyncio.apply()
import dataframe_image as dfi


# --------------------------------------
# Set working directory
# --------------------------------------
cwd = Path(os.getcwd())
print(cwd)

### Customized function

In [ ]:
# --------------------------------------
# Define functions
# --------------------------------------
def prony_response(comp_i, tdata, tau_0):
    # Compute the Prony series response.  
    # Args:
    # - comp_i : 1D array of prony coefficients
    # - tdata  : 1D array of time data
    # - tau_0  : 1D array of fixed retardation times
    # Returns:
    # - 1D numpy array with the response for each time point.
    comp_i = np.array(comp_i)
    return np.sum(comp_i) - np.sum(comp_i[:, None] * np.exp(-tdata / tau_0[:, None]), axis=0)

def fit_column(ydata, time, tau, maxiter=500):
    # Fit a compliance data series to the Prony series model.
    # Args:
    # - ydata  : 1D numpy array of compliance data
    # - time   : 1D numpy array of time values
    # - tau    : 1D numpy array of fixed retardation times
    # - maxiter: maximum number of iterations for the optimizer     
    # Returns:
    # - sol   : the optimized prony coefficients (accounting for sign flip)
    # - R2    : coefficient of determination for the fit

    """# If the average of the data is negative, flip the sign for a positive fit.
    if np.mean(ydata) < 0:
        ydata_fit = -ydata
        flip = -1
    else:
        ydata_fit = ydata
        flip = 1

    # Define the objective function: sum of squared relative errors (skipping the first point)
    def objective(comp_i):
        predicted = prony_response(comp_i, time, tau)
        return np.sum(((predicted[1:] - ydata_fit[1:]) / ydata_fit[1:])**2)

    N = len(tau)
    comp_0 = [0.5] * N  # initial guess for the prony coefficients
    result = minimize(objective, comp_0, bounds=[(1e-6, None)] * N, options={'maxiter': maxiter})
    sol = result.x * flip

    # Evaluate the fitted response and compute R²
    fitted_response = prony_response(sol, time, tau)
    SS_res = np.sum((ydata - fitted_response) ** 2)
    SS_tot = np.sum((ydata - np.mean(ydata)) ** 2)
    R2 = 1 - SS_res / SS_tot"""

    y = np.array(ydata, float)
    t = np.array(time, float)

    # Handle sign‐flip
    if y.mean() < 0:
        y_fit, flip = -y, -1
    else:
        y_fit, flip = y.copy(), 1

    N = len(tau)
    p0 = np.full(N, 0.5)
    lower = np.full(N, 1e-6)
    upper = np.full(N, np.inf)

    popt, _ = curve_fit(
        lambda t, *ci: prony_response(ci, t, tau),
        time, y_fit, p0=p0,
        bounds=(lower, upper), max_nfev=maxiter)
    sol = popt * flip

    # Calculate R-square coefficients
    fitted = prony_response(sol, t, tau)
    SS_res = np.sum((y - fitted) ** 2)
    SS_tot = np.sum((y - y.mean()) ** 2)
    R2 = 1 - SS_res/SS_tot

    return sol, R2

### Calculate Prony series coefficient $C_{i}^{-1}$ and $\gamma_{i}$ ratios

In [ ]:
# Set tissues
tissues = ['EW', 'TW', 'LW', 'GR']
# Set fixed retardation times [h]
tau = np.array([0.1, 1.0, 10.0, 100.0])
# Set components
pairs = ['11', '22', '33', '44', '55', '66', '12', '13', '23']

# Define column labels
prony_cols = [rf'$C_{{{i+1}}}$' for i in range(4)]
gamma_cols = [rf'$\gamma_{{{i+1}}}$' for i in range(4)]
columns = prony_cols + gamma_cols + [r'$R^2$']

# Map for subscript digits
sub_map = str.maketrans('0123456789', '₀₁₂₃₄₅₆₇₈₉')

def format_component(pair):
    # Example: pair = '11' → C₁₁⁻¹
    sub = pair.translate(sub_map)
    return f'C{sub}\u207B\u00B9'  # \u207B = superscript minus, \u00B9 = superscript 1

def format_column_C(i):
    # i = 1 → C₁⁻¹
    sub = str(i).translate(sub_map)
    return f'C{sub}\u207B\u00B9'

def format_column_gamma(i):
    # i = 1 → γ₁
    sub = str(i).translate(sub_map)
    return f'γ{sub}'

# Column names
prony_cols = [format_column_C(i+1) for i in range(4)]
gamma_cols = [format_column_gamma(i+1) for i in range(4)]
columns = prony_cols + gamma_cols# + ['R²']
row_labels = [format_component(p) for p in pairs]

# Loop over each tissue
all_results = {}
raw_data = {}
for tissue in tissues:
    # --------------------------------------
    # Load data
    # --------------------------------------
    tissue_folder = cwd / tissue
    elastic_path = tissue_folder / f"{tissue}_elastic_compliance_coeffs.csv"
    creep_path = tissue_folder / f"{tissue}_creep_compliance_coeffs.csv"
    C0inv = pd.read_csv(elastic_path)
    Jc = pd.read_csv(creep_path)
    # Preprocessing
    D_cols = [c for c in Jc.columns if c.startswith("D")]
    time_cols = [c for c in Jc.columns if c.startswith("time")]
    Jc[D_cols] = Jc[D_cols].subtract(C0inv.loc[0, D_cols], axis='columns')
    zero_row = {**{c: 0 for c in time_cols}, **{c: 0 for c in D_cols}}
    Jc = pd.concat([pd.DataFrame([zero_row]), Jc], ignore_index=True)

    # --------------------------------------
    # Fit Prony series
    # --------------------------------------
    results = []
    for pair in pairs:
        time_data = Jc[f"time{pair}"].values
        ydata = Jc[f"D{pair}"].values
        sol, R2 = fit_column(ydata, time_data, tau)
        baseline = C0inv[f"D{pair}"].iloc[0]
        gamma = baseline / np.array(sol)
        results.append(list(sol) + list(gamma))# + [R2])

    # --------------------------------------
    # Collect and show results
    # --------------------------------------
    df_results = pd.DataFrame(results, index=row_labels, columns=columns)
    df_results.columns.name = tissue
    all_results[tissue] = df_results
    raw_data[tissue] = {"Jc": Jc, "C0inv": C0inv}
    df_results = df_results.round(4)
    display(df_results)

    savepath = cwd / f"{tissue}_nlgeomOFF_prony_coeff_gamma.png"
    fig, ax = plt.subplots(figsize=(len(df_results.columns)*1.2, len(df_results)*0.4 + 1))
    ax.axis('off')
    formatted = df_results.applymap(lambda x: f"{x:.4f}").values

    tbl = ax.table(
        cellText = formatted,            # usa le stringhe formattate
        colLabels = df_results.columns,
        rowLabels = df_results.index,
        cellLoc   = 'center',
        loc       = 'center'
    )

    # Definizione dei colori pastello
    soft_grey   = '#f2f2f2'
    soft_blue   = '#cde7f0'
    soft_orange = '#fbe3d6'

    for (row, col), cell in tbl.get_celld().items():
        if col == -1:                    # colonna degli indici
            cell.set_facecolor(soft_grey)
        elif row == 0:                   # header row
            cell.set_facecolor(soft_blue if col < 4 else soft_orange)

    tbl.auto_set_font_size(False)
    tbl.set_fontsize(12)
    tbl.scale(1, 1.5)
    tbl.auto_set_column_width(col=list(range(len(df_results.columns))))

    for cell in tbl.get_celld().values():
        cell.set_edgecolor('#444444')
        cell.set_linewidth(0.5)

    plt.tight_layout()
    plt.savefig(savepath, dpi=300, bbox_inches='tight')
    plt.close(fig)

### Calculate average $\gamma_{i}$ and plot re-fitting of creep compliances $C_{jk}^{-1}$

In [ ]:
#tissues = ['EW', 'TW', 'LW']
# Prepare plot
fig, axes = plt.subplots(1, len(tissues), figsize=(10*len(tissues), 8))
fontsize = 24
colors = plt.cm.tab10(np.linspace(0, 1, len(pairs)))

def set_shared_grid(ax1, ax2):
    # 1) grab the current primary ticks
    primary_ticks = ax1.get_yticks()
    # 2) compute the uniform step
    step = primary_ticks[1] - primary_ticks[0]
    # 3) extend the primary limits by one step on each end
    new_bottom = primary_ticks[0] - step
    new_top = primary_ticks[-1] + step
    ax1.set_ylim(new_bottom, new_top)
    # 4) keep the same gridlines at the primary_ticks
    ax1.set_yticks(primary_ticks)
    # 5) now compute secondary ticks by slicing its data‐range into len(primary_ticks) points
    ax2.set_ylim(bottom=0)
    smin, smax = ax2.get_ylim()
    secondary_ticks = np.linspace(0, smax, len(primary_ticks)+2)
    ax2.set_yticks(secondary_ticks)
    ax2.set_yticklabels([str(round(t,1)) for t in secondary_ticks])
    for i in [0,-1]: ax2.get_yticklabels()[i].set_visible(False)

    return ax1, ax2

# Loop over each tissue
gamma_summary = []
handles = []
for idx, tissue in enumerate(tissues):

    df = all_results[tissue]

    # --------------------------------------
    # Calculate mean gammas
    # --------------------------------------   
    gamma_means = df[gamma_cols].mean()
    gamma_means.name = tissue
    gamma_summary.append(gamma_means)

    # --------------------------------------
    # Plot fitting with mean gammas
    # --------------------------------------
    Jc = raw_data[tissue]['Jc']
    C0inv = raw_data[tissue]['C0inv']
    ax = axes[idx] if len(tissues) > 1 else axes
    handles = []
    for jdx, pair in enumerate(pairs):
        # Get raw data
        tdata = Jc[f"time{pair}"].values
        ydata = Jc[f"D{pair}"].values
        baseline = C0inv[f"D{pair}"].iloc[0]
        # Calculate gamma ratio and fit Prony series
        C_new = baseline / gamma_means.values
        t_dense = np.linspace(tdata.min(), tdata.max(), 100)
        y_fit = sum(C_new[k] * (1 - np.exp(-t_dense / tau[k])) for k in range(len(tau)))
        # Plot all components on primary y-axis except 66-component (RT-shear) on secondary y-axis
        color = colors[jdx]
        if pair == '66':
            ax2 = ax.twinx()
            ax2.grid(False)
            ax2.plot(tdata, ydata, 'o', color=color)
            ax2.plot(t_dense, y_fit, '--', color=color)
            ax2.set_ylabel(r'$C_{66}^{-1}$ [1/MPa]', color=color, fontsize=fontsize, rotation=270, labelpad=35)
            ax2.yaxis.label.set_color(color)
        else:
            label = format_component(pair)
            ax.plot(tdata, ydata, 'o', color=color, label=label)
            ax.plot(t_dense, y_fit, '--', color=color)

    # Force shared grid
    ax, ax2 = set_shared_grid(ax, ax2)
    ax2.tick_params(axis='y', labelsize=fontsize)

    # Set legend
    if idx == len(tissues) - 1:
        handles, labels = ax.get_legend_handles_labels()
        # Create the final fit handle
        label = r'fit with $\overline{\gamma_i}$'
        fit_line = Line2D([], [], color='black', linestyle='--', label=label)
        handles.append(fit_line)
        labels.append(label)
        # Set legend with title + all handles + fit handle
        leg = ax.legend(handles, labels, title='Sim. creep compl.', title_fontsize=fontsize, 
            fontsize=fontsize, loc='center left', bbox_to_anchor=(1.25, 0.5))

    # Customize axes
    ax.set_title(tissue, fontsize=fontsize)
    ax.set_xlabel('Time [h]', fontsize=fontsize)
    ax.set_xlim(left=0, right=max(tdata))
    ax.set_ylabel(r'$C_{jk}^{-1}$ [1/MPa]', fontsize=fontsize, labelpad=-10)
    ax.tick_params(labelsize=fontsize)
    ax.grid(alpha=0.5)
    ax.set_axisbelow(True)

# Show plot
leg.get_title().set_ha('left')
plt.tight_layout()
plt.show()

# Show summary table
df_gamma_summary = pd.DataFrame(gamma_summary, index=tissues, columns=gamma_cols)
df_gamma_summary.columns.name ='Tissue'
display(df_gamma_summary.round(4))

# Save plot
savepath = cwd / "Y-RVE_nlgeomON_creep_compl.png"
fig.savefig(savepath, dpi=300, bbox_inches='tight')

### Plot color mapping of relative error of $\gamma_{i}$ wrt the respective mean value

In [ ]:
# Prepare plot
fig, axs = plt.subplots(1, len(tissues), figsize=(6 * len(tissues), 6))
axs = axs.flatten()
fontsize = 24

# Mapping from 'D11' to '$C_{11}^{-1}$'
def format_label(label):
    num = label[1:]
    return rf'$C_{{{num}}}^{{-1}}$'

# First, compute global vmin/vmax
all_diffs = []
for tissue in tissues:
    dfr = all_results[tissue]
    gm = dfr[gamma_cols].mean()
    diff = (dfr[gamma_cols] - gm) / gm * 100
    all_diffs.append(diff.values)
all_diffs = np.vstack(all_diffs)
vmin, vmax = np.nanmin(all_diffs), np.nanmax(all_diffs)

# Now plot each subplot with the same scale
for i, tissue in enumerate(tissues):
    ax = axs[i]
    dfr = all_results[tissue]

    gamma_means = dfr[gamma_cols].mean()
    diff_gamma  = (dfr[gamma_cols] - gamma_means) / gamma_means * 100

    im = ax.imshow(
        diff_gamma.values,
        aspect='auto',
        cmap='coolwarm',
        vmin=vmin,
        vmax=vmax)

    ax.set_xticks(np.arange(len(gamma_cols)))
    ax.set_xticklabels(gamma_cols, rotation=45, fontsize=fontsize)

    ax.set_yticks(np.arange(len(diff_gamma.index)))
    if i == 0:
        ax.set_yticklabels([format_label(lbl) for lbl in diff_gamma.index], fontsize=fontsize)
    else:
        ax.set_yticklabels([])

    ax.set_title(tissue, fontsize=fontsize)

    # Annotate max and min
    vals = diff_gamma.values
    max_idx = np.unravel_index(np.nanargmax(vals), vals.shape)
    min_idx = np.unravel_index(np.nanargmin(vals), vals.shape)
    max_val = vals[max_idx]
    min_val = vals[min_idx]
    ax.text(
        max_idx[1], max_idx[0], f'{max_val:.1f}%', 
        ha='center', va='center', fontsize=fontsize - 4,
        color='white' if abs(max_val) > abs(vmin) else 'black')
    ax.text(
        min_idx[1], min_idx[0], f'{min_val:.1f}%', 
        ha='center', va='center', fontsize=fontsize - 4,
        color='white' if abs(min_val) > abs(vmin) else 'black')

# 1) Make room at the bottom
fig.tight_layout()
fig.subplots_adjust(bottom=0.2)

M = max(abs(vmin), abs(vmax))

# and *after* your last imshow (or instead of passing vmin/vmax there):
im.set_clim(-M, M)

# 1) choose new bar dimensions
bar_width  = 0.5   # 50% of figure width
bar_height = 0.03  # 3% of figure height

# 2) center horizontally: left = (1 - width)/2
left = (1.03 - bar_width) / 2.0
bottom = 0.05      # keep it 5% from bottom

# 3) create the axes and colorbar
cbar_ax = fig.add_axes([left, bottom, bar_width, bar_height])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
cbar.set_label(r'$(\gamma_i - \bar{\gamma}_i)/\bar{\gamma}_i$ [%]', fontsize=fontsize)
cbar.ax.tick_params(labelsize=fontsize)

plt.show()

# Save plot
savepath = cwd / "Y-RVE_nlgeomOFF_gamma_colormap.png"
fig.savefig(savepath, dpi=300, bbox_inches='tight')
